# Exploratory analysis
## Using simulated data

In [10]:
import sgkit as sg
import xarray as xr
import numpy as np

In [15]:
sim_data = sg.simulate_genotype_call_dataset(
    n_variant=1000, n_sample=250, n_contig=23, missing_pct=.1
)

sim_data

<xarray.Dataset> Size: 1MB
Dimensions:             (contigs: 23, variants: 1000, alleles: 2, samples: 250,
                         ploidy: 2)
Dimensions without coordinates: contigs, variants, alleles, samples, ploidy
Data variables:
    contig_id           (contigs) <U2 184B '0' '1' '2' '3' ... '20' '21' '22'
    variant_contig      (variants) int64 8kB 0 0 0 0 0 0 0 ... 22 22 22 22 22 22
    variant_position    (variants) int64 8kB 0 1 2 3 4 5 6 ... 37 38 39 40 41 42
    variant_allele      (variants, alleles) |S1 2kB b'G' b'A' b'T' ... b'A' b'T'
    sample_id           (samples) <U4 4kB 'S0' 'S1' 'S2' ... 'S248' 'S249'
    call_genotype       (variants, samples, ploidy) int8 500kB 0 0 1 0 ... 0 0 1
    call_genotype_mask  (variants, samples, ploidy) bool 500kB False ... False
Attributes:
    contigs:  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', ...
    source:   sgkit-0.10.0

In [6]:
sg.display_genotypes(sim_data, max_variants=8, max_samples=8)

samples,S0,S1,S2,S3,...,S246,S247,S248,S249
variants,,,,,,,,,
0,0/0,1/0,1/0,0/1,...,1/1,0/0,1/1,1/0
1,0/0,0/1,1/1,1/1,...,0/1,1/0,0/.,0/0
2,1/1,0/1,1/0,0/1,...,0/0,1/1,0/1,./1
3,1/0,1/0,1/0,1/0,...,1/1,1/0,0/1,1/.
...,...,...,...,...,...,...,...,...,...
39,0/.,1/1,1/1,0/0,...,./.,0/0,1/0,./0
40,0/0,./0,1/0,1/1,...,1/.,1/0,1/1,./1
41,0/1,0/1,1/1,1/0,...,0/0,0/0,0/1,./1
42,1/0,0/1,1/0,1/1,...,0/1,1/0,0/0,0/1


### Cohorts
Cohorts are groups of samples (think population). They are defined by mapping a cohort index onto samples

In [12]:
sim_data_cohort = sg.simulate_genotype_call_dataset(n_variant=100, n_sample=10)
sim_data_cohort["sample_cohort"] = xr.DataArray(np.array([-1, 0, 1, 1, 1, 1, 0, 2, 2, 2]), dims="samples")
sim_data_cohort

<xarray.Dataset> Size: 6kB
Dimensions:             (contigs: 1, variants: 100, alleles: 2, samples: 10,
                         ploidy: 2)
Dimensions without coordinates: contigs, variants, alleles, samples, ploidy
Data variables:
    contig_id           (contigs) <U1 4B '0'
    variant_contig      (variants) int64 800B 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    variant_position    (variants) int64 800B 0 1 2 3 4 5 ... 94 95 96 97 98 99
    variant_allele      (variants, alleles) |S1 200B b'T' b'T' ... b'T' b'G'
    sample_id           (samples) <U2 80B 'S0' 'S1' 'S2' 'S3' ... 'S7' 'S8' 'S9'
    call_genotype       (variants, samples, ploidy) int8 2kB 0 0 1 0 ... 0 1 1 1
    call_genotype_mask  (variants, samples, ploidy) bool 2kB False ... False
    sample_cohort       (samples) int64 80B -1 0 1 1 1 1 0 2 2 2
Attributes:
    contigs:  ['0']
    source:   sgkit-0.10.0

Using `sgkit.Fst()` produces stats for pairs (think pairwise FST by population).

In [ ]:
sim_data_cohort = sg.window_by_variant(sim_data_cohort, size=20)

sim_data_cohort = sg.Fst(sim_data_cohort)

cohort_names = ["Africa", "Asia", "Europe"]

sim_data_cohort = sim_data_cohort.assign_coords({"cohorts_0": cohort_names, "cohorts_1": cohort_names})

sim_data_cohort.stat_Fst.sel(cohorts_0="Africa", cohorts_1="Asia").values

array([ 0.00823045, -0.02510823,  0.04372041,  0.12240725, -0.11305475])

## Using my test data
`test/ProjTaxaTest.vcf`

In [22]:
data = sg.load_dataset("../test/ProjTaxaTest.vcz")
data

<xarray.Dataset> Size: 950kB
Dimensions:                  (variants: 1000, samples: 16, FORMAT_AD_dim: 5,
                              genotypes: 15, FORMAT_SB_dim: 4, ploidy: 2,
                              contigs: 2301, filters: 2,
                              region_index_values: 2, region_index_fields: 6,
                              alt_alleles: 4, alleles: 5)
Dimensions without coordinates: variants, samples, FORMAT_AD_dim, genotypes,
                                FORMAT_SB_dim, ploidy, contigs, filters,
                                region_index_values, region_index_fields,
                                alt_alleles, alleles
Data variables: (12/47)
    call_AD                  (variants, samples, FORMAT_AD_dim) int8 80kB dask.array<chunksize=(1000, 16, 5), meta=np.ndarray>
    call_DP                  (variants, samples) int8 16kB dask.array<chunksize=(1000, 16), meta=np.ndarray>
    call_GQ                  (variants, samples) int8 16kB dask.array<chunksize=(1000, 16), meta=np.ndarray>
    call_MIN_DP              (variants, samples) int8 16kB dask.array<chunksize=(1000, 16), meta=np.ndarray>
    call_PL                  (variants, samples, genotypes) int16 480kB dask.array<chunksize=(1000, 16, 15), meta=np.ndarray>
    call_SB                  (variants, samples, FORMAT_SB_dim) int8 64kB dask.array<chunksize=(1000, 16, 4), meta=np.ndarray>
    ...                       ...
    variant_filter           (variants, filters) bool 2kB dask.array<chunksize=(1000, 2), meta=np.ndarray>
    variant_id               (variants) object 8kB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_id_mask          (variants) bool 1kB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_length           (variants) int8 1kB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_position         (variants) int32 4kB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_quality          (variants) float32 4kB dask.array<chunksize=(1000,), meta=np.ndarray>
Attributes:
    source:                bio2zarr-0.1.6
    vcf_meta_information:  [['fileformat', 'VCFv4.1'], ['ALT', '<ID=NON_REF,D...
    vcf_zarr_version:      0.4